<a href="https://colab.research.google.com/github/Marriyam/TEVTA-AI-ML/blob/main/using_fuel_Cars_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:

car_data = pd.read_csv('/content/car_dataset_1000_rows.csv')

car_data['car_prices_in_rup'] = car_data['car_prices_in_rup'].str.replace(' Lakh','').astype(float) * 100000
car_data['kms_driven'] = car_data['kms_driven'].str.replace(' kms','').str.replace(',','').astype(int)
car_data['engine'] = car_data['engine'].str.replace(' cc','').str.replace(',','').astype(int)
car_data['ownership'] = car_data['ownership'].str.extract('(\\d+)').astype(int)
car_data['manufacture'] = pd.to_datetime(car_data['manufacture'], errors='coerce')
car_data['manufacture_year'] = car_data['manufacture'].dt.year
car_data['fuel_type']=LabelEncoder().fit_transform(car_data['fuel_type'])
car_data['transmission']=LabelEncoder().fit_transform(car_data['transmission'])
car_data=car_data.drop(['car_name'], axis=1)

ohe = pd.get_dummies(car_data['transmission'], prefix='transmission')
car_data=car_data.drop([ 'transmission'], axis=1)
car_data = pd.concat([car_data, ohe], axis=1)
car_data

,car_prices_in_rup,kms_driven,fuel_type,ownership,engine,seats,manufacture,manufacture_year,transmission_0,transmission_1
0,4433000.0,52618,2,1,1207,5,2023-05-05,2023,True,False
1,1866000.0,42295,1,3,2204,4,2022-09-29,2022,False,True
2,1983000.0,34289,0,1,1106,5,2015-11-08,2015,False,True
3,3863000.0,90279,2,2,2453,6,2017-01-20,2017,False,True
4,3159000.0,8988,1,3,2016,5,2023-02-11,2023,False,True
...,...,...,...,...,...,...,...,...,...,...
995,2264000.0,94437,1,3,2006,6,2016-08-21,2016,True,False
996,4757000.0,1263,2,3,2845,4,2023-11-21,2023,True,False
997,1240000.0,14088,3,1,1309,6,2016-04-03,2016,False,True
998,3405000.0,12041,1,3,2566,4,2022-02-09,2022,False,True


In [ ]:

current_date = pd.to_datetime("2023-10-01")
car_data['car_age'] = (current_date - car_data['manufacture']).dt.days // 365
car_data['price_per_km'] = car_data['car_prices_in_rup'] / car_data['kms_driven']
car_data['is_new'] = (car_data['car_age'] <= 1).astype(int)
car_data['price_category'] = pd.qcut(car_data['car_prices_in_rup'], q=3, labels=['Low', 'Medium', 'High'])
car_data['price_category'] =LabelEncoder().fit_transform(car_data['price_category'])

car_data

,car_prices_in_rup,kms_driven,fuel_type,ownership,engine,seats,manufacture,manufacture_year,transmission_0,transmission_1,car_age,price_per_km,is_new,price_category
0,4433000.0,52618,2,1,1207,5,2023-05-05,2023,True,False,0,84.248736,1,0
1,1866000.0,42295,1,3,2204,4,2022-09-29,2022,False,True,1,44.118690,1,2
2,1983000.0,34289,0,1,1106,5,2015-11-08,2015,False,True,7,57.831958,0,2
3,3863000.0,90279,2,2,2453,6,2017-01-20,2017,False,True,6,42.789575,0,0
4,3159000.0,8988,1,3,2016,5,2023-02-11,2023,False,True,0,351.468625,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2264000.0,94437,1,3,2006,6,2016-08-21,2016,True,False,7,23.973654,0,2
996,4757000.0,1263,2,3,2845,4,2023-11-21,2023,True,False,-1,3766.429137,1,0
997,1240000.0,14088,3,1,1309,6,2016-04-03,2016,False,True,7,88.018171,0,1
998,3405000.0,12041,1,3,2566,4,2022-02-09,2022,False,True,1,282.783822,1,0


In [ ]:
car_data.columns.to_list()

['car_prices_in_rup',
 'kms_driven',
 'fuel_type',
 'ownership',
 'engine',
 'seats',
 'manufacture',
 'manufacture_year',
 'transmission_0',
 'transmission_1',
 'car_age',
 'price_per_km',
 'is_new',
 'price_category']

In [ ]:
from re import X
cols = ['car_prices_in_rup',
 'kms_driven',
 'ownership',
 'engine',
 'seats',
 'manufacture_year',
 'transmission_0',
 'transmission_1',
 'car_age',
 'price_per_km',
 'is_new',
 'price_category']


X_train, X_test, y_train, y_test = train_test_split(car_data[cols], car_data['fuel_type'], test_size=0.3, random_state=42)
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

dt_importances = dt_classifier.feature_importances_

dt_importance_df = pd.DataFrame({'Feature': cols, 'Importance': dt_importances})
dt_importance_df = dt_importance_df.sort_values(by='Importance', ascending=False)

print("Decision Tree Feature Importance:")
print(dt_importance_df)

Decision Tree Feature Importance:
              Feature  Importance
0   car_prices_in_rup    0.186945
9        price_per_km    0.185465
3              engine    0.181177
1          kms_driven    0.156447
8             car_age    0.067598
5    manufacture_year    0.059765
4               seats    0.057290
2           ownership    0.044821
11     price_category    0.029420
6      transmission_0    0.018013
7      transmission_1    0.009726
10             is_new    0.003334


In [ ]:
X=car_data.drop(['fuel_type', 'is_new', 'transmission_1', 'transmission_0', 'price_category', 'ownership', 'seats', 'manufacture_year', 'car_age'], axis=1)
y=car_data['fuel_type']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
models = {
    'Random Forest Classifier': RandomForestClassifier(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'Decision Tree Classifier': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42)
}

param_grids = {
    'Random Forest Classifier': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    'Support Vector Machine': {
        'C': [100, 500],
        'kernel': ['rbf'],
        'gamma': ['scale', 'auto', 1, 10]
    },
    'Decision Tree Classifier': {
        'max_depth': [None],
        'min_samples_split': [2]
    },
    'Logistic Regression': {
        'C': [0.1, 1, 10],
    }
}


In [ ]:
best_classifiers = {}
classification_results = {}

for model_name, model in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=5,  scoring='accuracy', n_jobs=-1, verbose=1 )
    grid_search.fit(X_train_scaled, y_train)
    best_classifiers[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}\n")

for model_name, model in best_classifiers.items():
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred) * 100
    precision = precision_score(y_test, y_pred, average='weighted') * 100
    recall = recall_score(y_test, y_pred, average='weighted') * 100
    f1 = f1_score(y_test, y_pred, average='weighted') * 100
    classification_results[model_name] = {
        'Accuracy (%)': round(accuracy, 2),
        'Precision (%)': round(precision, 2),
        'Recall (%)': round(recall, 2),
        'F1-Score (%)': round(f1, 2)
    }

classification_results_df = pd.DataFrame(classification_results).T
classification_results_df = classification_results_df.sort_values(by='Accuracy (%)', ascending=False)
print("### Classification Model Performance:\n")
print(classification_results_df)

for model_name, model in best_classifiers.items():
    y_pred = model.predict(X_test_scaled)
    print(f"\n### {model_name} Classification Report:\n")
    print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best parameters for Random Forest Classifier: {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100}

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters for Support Vector Machine: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters for Decision Tree Classifier: {'max_depth': None, 'min_samples_split': 2}

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Logistic Regression: {'C': 1}

### Classification Model Performance:

                          Accuracy (%)  Precision (%)  Recall (%)  \
Logistic Regression              27.00          26.94       27.00   
Random Forest Classifier         25.33          25.74       25.33   
Decision Tree Classifier         25.33          25.86       25.33   
Support Vector Machine           23.33          24.12       23.33   

                          F1-Score (%)  
Logistic Regression              26.47  
Random Forest Cla